In [65]:
from TTS.api import TTS
from pydub import AudioSegment
import os
from tqdm import tqdm

In [ ]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True)

In [71]:
def split_text(text, max_len=200):
    paragraphs = text.split("\n")
    chunks = []
    for paragraph in paragraphs:
        sentences = paragraph.split(".")
        chunk = ""
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) == 0:
                continue
            sentence += "."
            if len(chunk) + len(sentence) <= max_len:
                chunk += " " + sentence
            else:
                if len(sentence) > max_len:
                    for i in range(0, len(sentence), max_len):
                        sub_sentence = sentence[i:i+max_len]
                        if sub_sentence[-1] != ".":
                            sub_sentence += "."
                        chunks.append(sub_sentence)
                else:
                    chunks.append(chunk.strip())
                    chunk = sentence
        if chunk:
            chunks.append(chunk.strip())
    return chunks

def gen_speech(text, name='output', speaker="Ana Florence", language="en", out_fmt="mp3"):
    audio = AudioSegment.empty()
    outfilename = f"{name}_{language}"
    for ind, chunk in enumerate(tqdm(split_text(text))):
        file_path=f"./processed_audio/{outfilename}_chunk_{ind}.wav"
        tts.tts_to_file(text=chunk,
            file_path=file_path,
            speaker=speaker,
            language=language)
        
        audio_chunk = AudioSegment.from_file(file_path)
        audio += audio_chunk
    
    audio.export(f"./processed_audio/{outfilename}.{out_fmt}", format=out_fmt)
    
    # remove the chunk audio files
    # for ind in range(len(split_text(text))):
    #     file_path=f"{name}_{language}_chunk_{ind}.wav"
    #     os.remove(file_path)
    return audio

In [75]:
with open("./inputs/meluha_text_en.txt", "r") as f:
    text_en = f.read()

gen_speech(text_en, name="meluha", language="en", speaker="Ana Florence")

  0%|          | 0/9 [00:00<?, ?it/s]

 > Text splitted to sentences.
['Shiva gazed at the orange sky.', 'The clouds hovering above Mansarovar had just parted to reveal the setting sun.', 'The brilliant giver of life was calling it a day once again.']


 11%|█         | 1/9 [00:20<02:44, 20.59s/it]

 > Processing time: 20.568287134170532
 > Real-time factor: 1.5986504261902186
 > Text splitted to sentences.
['But the sunset!', 'He tried never to miss the sunset!', 'On any other day, Shiva would have taken in the vista — the sun and the immense lake against the magnificent backdrop of the Himalayas stretching as .']


 22%|██▏       | 2/9 [00:46<02:44, 23.45s/it]

...

 > Processing time: 9.605350971221924
 > Real-time factor: 1.6252147706832676
 > Text splitted to sentences.
['On a normal day, Shiva would have smiled at the memory from a cheerful past that had been overwhelmed by the angst of the present.']


 89%|████████▉ | 8/9 [01:54<00:13, 13.00s/it]

 > Processing time: 13.036885023117065
 > Real-time factor: 1.6884181160119545
 > Text splitted to sentences.
['But today, he turned back towards his village without any hint of joy.']


100%|██████████| 9/9 [02:03<00:00, 13.75s/it]

 > Processing time: 8.94881296157837
 > Real-time factor: 1.6123130948719033


In [76]:
with open("./inputs/meluha_text_hi.txt", "r") as f:
    text_hi = f.read()

gen_speech(text_hi, name="meluha", language="hi", speaker="Ana Florence")

  0%|          | 0/6 [00:00<?, ?it/s]

 > Text splitted to sentences.
['शिव ने नारंगी आकाश की ओर देखा। मानसरोवर के ऊपर मंडरा रहे बादल अस्त होते सूर्य को दिखाने के लिए अभी-अभी छंट चुके थे। जीवन का प्रतिभाशाली दाता एक बार फिर अपने दिन का आह्वान कर रहा था। शिव ने अपनी इक्कीस.']


 17%|█▋        | 1/6 [00:27<02:15, 27.11s/it]

 > Processing time: 27.083282947540283
 > Real-time factor: 1.7291706885373617
 > Text splitted to sentences.
['वर्ष की आयु में कुछ सूर्योदय देखे थे। लेकिन सूर्यास्त!', 'उसने कोशिश की कि कभी भी सूर्यास्त न छूटे!', 'किसी भी अन्य दिन, शिव ने सूर्य और हिमालय की शानदार पृष्ठभूमि के सामने विशाल झील का दृश्य देखा होगा, जह.']


 33%|███▎      | 2/6 [01:00<02:04, 31.03s/it]

...

 > Processing time: 28.923183917999268
 > Real-time factor: 1.7347679347605318
 > Text splitted to sentences.
['तरह याद थे। उन्होंने झील की सतह से उछलकर आने वाले कंकड़ फेंकने की कला में महारत हासिल कर ली थी। उन्होंने अभी भी अपने समुदाय में सबसे अधिक बाउंस: सत्रह का रिकॉर्ड कायम रखा है।.']


 83%|████████▎ | 5/6 [02:01<00:23, 23.94s/it]

 > Processing time: 24.294398069381714
 > Real-time factor: 1.6874511032390844
 > Text splitted to sentences.
['एक सामान्य दिन में, शिव अपने हँसमुख अतीत को याद करके मुस्कुरा देते थे जो वर्तमान के संताप से अभिभूत हो गया था। लेकिन आज वह बिना किसी खुशी के वापस अपने गांव की ओर मुड़ गया।.']


100%|██████████| 6/6 [02:24<00:00, 24.06s/it]

 > Processing time: 22.63956093788147
 > Real-time factor: 1.691156426772069
